# Imports

In [20]:
import pandas as pd

import math
from math import pi, pow, sin, cos, asin, sqrt, floor
from scipy import stats
import numpy as np
from pyproj import Proj

# 2020 Election Data

In [25]:
data_2020 = pd.read_excel('/Users/gracefujinaga/Documents/Northwestern/MSDS_460/redistricting/ge2020cty_638659169862046352.xls')

# only keep if it is president or senator
data_2020 = data_2020[data_2020['OfficeName'].str.contains('president|senator', case=False)]

data_2020.columns
data_2020.drop(columns = ['VoteFor', 'BallotGroup', 'OfficeSequence', 'PartyAbbrev', 'CanAffilCommit'], inplace=True)
data_2020.reset_index(drop=True, inplace=True)

# drop if there are zero votes
data_2020 = data_2020[(data_2020 != 0).all(axis=1)]

# check na values
data_2020.isna().sum()

WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero


Election         0
OfficeName       0
CandidateID      0
CanFirstName     0
CanLastName      0
County           0
Votes            0
PartyName       69
dtype: int64

In [ ]:
data_2020

In [27]:
df = data_2020

In [37]:
# now group data to get proportion by county and by state
# Group by County and Party, and calculate total votes
county_grouped = df.groupby(['County', 'PartyName'])['Votes'].sum().reset_index()

# Calculate total votes by County
total_votes_by_county = county_grouped.groupby('County')['Votes'].sum().reset_index()
total_votes_by_county.rename(columns={'Votes': 'TotalVotes'}, inplace=True)

# Merge to get total votes with party votes
merged = pd.merge(county_grouped, total_votes_by_county, on='County')

# Calculate the proportion of votes by party
merged['Proportion'] = merged['Votes'] / merged['TotalVotes']
merged.drop(columns=['Votes', 'TotalVotes'], inplace=True)
proportion_by_county = merged.reset_index(drop=True)

proportion_by_county.to_csv('/Users/gracefujinaga/Documents/Northwestern/MSDS_460/redistricting/proportion_by_county_2020.csv', index=False)
proportion_by_county

# Step 5: Group by State and Party and calculate total votes (if needed)
# Assuming we have a column for State (you might need to modify your schema)
# total_votes_by_state = df.groupby(['State', 'PartyName'])['Votes'].sum().reset_index()
# total_votes_by_state['Proportion'] = total_votes_by_state['Votes'] / total_votes_by_state.groupby('State')['Votes'].transform('sum')



,County,PartyName,Proportion
0,ADAMS,AMERICAN SOLIDARITY PARTY,0.000958
1,ADAMS,DEMOCRATIC,0.261532
2,ADAMS,GREEN,0.003998
3,ADAMS,LIBERTARIAN,0.013116
4,ADAMS,PARTY FOR SOCIALISM AND LIBERATION,0.000374
...,...,...,...
704,WOODFORD,GREEN,0.005826
705,WOODFORD,LIBERTARIAN,0.016139
706,WOODFORD,PARTY FOR SOCIALISM AND LIBERATION,0.000399
707,WOODFORD,REPUBLICAN,0.680613


In [41]:
# Group by County and Party, and calculate total votes
grouped = df.groupby(['PartyName'])['Votes'].sum().reset_index()
grouped

# Calculate total votes by County
total_votes = grouped['Votes'].sum()

# add total votes column
grouped['TotalVotes'] = total_votes

# Calculate the proportion of votes by party
grouped['Proportion'] = grouped['Votes'] / grouped['TotalVotes']
grouped.drop(columns=['Votes', 'TotalVotes'], inplace=True)
grouped.sort_values(by='Proportion', ascending=False, inplace=True)
grouped.reset_index(drop=True, inplace=True)
grouped.to_csv('/Users/gracefujinaga/Documents/Northwestern/MSDS_460/redistricting/proportion2020.csv', index=False)
grouped

,PartyName,Proportion
0,DEMOCRATIC,0.562462
1,REPUBLICAN,0.397153
2,WILLIE WILSON PARTY,0.019804
3,LIBERTARIAN,0.011849
4,GREEN,0.007266
5,AMERICAN SOLIDARITY PARTY,0.000796
6,PARTY FOR SOCIALISM AND LIBERATION,0.000670


# 2022 Election Data

In [45]:
df = pd.read_excel('/Users/gracefujinaga/Documents/Northwestern/MSDS_460/redistricting/2022gecty_638659169289081376.xls')

# only keep if it is president or senator
df = df[df['OfficeName'].str.contains('president|senator', case=False)]

df.columns
df.drop(columns = ['VoteFor', 'BallotGroup', 'OfficeSequence', 'PartyAbbrev', 'CanAffilCommit'], inplace=True)
df.reset_index(drop=True, inplace=True)

# drop if there are zero votes
df = df[(df != 0).all(axis=1)]

# check na values
df.isna().sum()

df

WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero


,Election,OfficeName,CandidateID,CanFirstName,CanLastName,County,Votes,PartyName
0,GE 2022,UNITED STATES SENATOR,31007,BILL,REDPATH,OGLE,377,LIBERTARIAN
1,GE 2022,UNITED STATES SENATOR,31007,BILL,REDPATH,LEE,204,LIBERTARIAN
2,GE 2022,UNITED STATES SENATOR,31007,BILL,REDPATH,POPE,24,LIBERTARIAN
3,GE 2022,UNITED STATES SENATOR,31007,BILL,REDPATH,TAZEWELL,1067,LIBERTARIAN
4,GE 2022,UNITED STATES SENATOR,31007,BILL,REDPATH,SHELBY,186,LIBERTARIAN
...,...,...,...,...,...,...,...,...
489,GE 2022,UNITED STATES SENATOR,31007,BILL,REDPATH,DeKALB,747,LIBERTARIAN
490,GE 2022,UNITED STATES SENATOR,31007,BILL,REDPATH,LAKE,3295,LIBERTARIAN
491,GE 2022,UNITED STATES SENATOR,31007,BILL,REDPATH,COOK,20282,LIBERTARIAN
492,GE 2022,UNITED STATES SENATOR,31007,BILL,REDPATH,WINNEBAGO,1646,LIBERTARIAN


In [46]:
# now group data to get proportion by county and by state
# Group by County and Party, and calculate total votes
county_grouped = df.groupby(['County', 'PartyName'])['Votes'].sum().reset_index()

# Calculate total votes by County
total_votes_by_county = county_grouped.groupby('County')['Votes'].sum().reset_index()
total_votes_by_county.rename(columns={'Votes': 'TotalVotes'}, inplace=True)

# Merge to get total votes with party votes
merged = pd.merge(county_grouped, total_votes_by_county, on='County')

# Calculate the proportion of votes by party
merged['Proportion'] = merged['Votes'] / merged['TotalVotes']
merged.drop(columns=['Votes', 'TotalVotes'], inplace=True)
proportion_by_county = merged.reset_index(drop=True)

proportion_by_county.to_csv('/Users/gracefujinaga/Documents/Northwestern/MSDS_460/redistricting/proportion_by_county_2022.csv', index=False)
proportion_by_county


,County,PartyName,Proportion
0,ADAMS,DEMOCRATIC,0.250401
1,ADAMS,LIBERTARIAN,0.014076
2,ADAMS,REPUBLICAN,0.735523
3,ALEXANDER,DEMOCRATIC,0.447569
4,ALEXANDER,LIBERTARIAN,0.007616
...,...,...,...
301,WINNEBAGO,LIBERTARIAN,0.018704
302,WINNEBAGO,REPUBLICAN,0.488307
303,WOODFORD,DEMOCRATIC,0.281633
304,WOODFORD,LIBERTARIAN,0.018550


In [47]:
# Group by County and Party, and calculate total votes
grouped = df.groupby(['PartyName'])['Votes'].sum().reset_index()
grouped

# Calculate total votes by County
total_votes = grouped['Votes'].sum()

# add total votes column
grouped['TotalVotes'] = total_votes

# Calculate the proportion of votes by party
grouped['Proportion'] = grouped['Votes'] / grouped['TotalVotes']
grouped.drop(columns=['Votes', 'TotalVotes'], inplace=True)
grouped.sort_values(by='Proportion', ascending=False, inplace=True)
grouped.reset_index(drop=True, inplace=True)
grouped.to_csv('/Users/gracefujinaga/Documents/Northwestern/MSDS_460/redistricting/proportion2022.csv', index=False)
grouped

,PartyName,Proportion
0,DEMOCRATIC,0.568240
1,REPUBLICAN,0.415007
2,LIBERTARIAN,0.016754


# Getting distance between counties

# Getting Dictionary For Population

In [ ]:
pop_data = pd.read_excel('/Users/gracefujinaga/Documents/Northwestern/MSDS_460/redistricting/MSDS460_Assignment_3_Spreadsheet.xlsx')
county_population = df.set_index('county')['pop2024'].to_dict()
county_population